In [1]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [2]:
from mindsponge import Sponge
from mindsponge import ForceField
from mindsponge.optimizer import SteepestDescent
from mindsponge.control import VelocityVerlet
from mindsponge.callback import WriteH5MD, RunInfo
from mindsponge.control import Langevin
from mindsponge import set_global_units
from mindsponge import Protein
from mindsponge import DynamicUpdater
from mindsponge.function import VelocityGenerator

## 单位设置

这里我们提到的全局单位设置，主要是指输入输出的长度单位和能量单位，在MindSponge的计算过程中会自动根据默认单位进行计算，再将返回的结果转换成用户定义的单位。常用的长度单位是nm和A，一般在PDB文件里面是用A为单位。能量单位比较常用的是kJ/mol和kcal/mol。

In [3]:
set_global_units('nm', 'kj/mol')

## 蛋白质案例

我们提供了一些简单的蛋白质案例以供测试，一般用pdb(Protein Data Bank)格式来存储蛋白质文件。下述案例就是一个不含H原子的pdb文件（该构象来自于MEGAProtein的预测结果）。

![](docs/case2-1.png)

在使用MindSponge构建Protein对象时，会自动为其在相对合理的位置补充H原子（如下图所示）。而这一过程之后，最好对其进行一次能量最小化的操作，以确保得到的构象中不会存在太多的冲突，比如两个原子距离太近等。

![](docs/case2-3.png)

如果使用常用的蛋白质可视化算法NewCartoon来展示的话，可以更清晰的看到这个构象中所存在的$\alpha$螺旋和$\beta$折叠等区域：

![](docs/case2-2.png)

In [4]:
pdb_name = 'case2.pdb'
system = Protein(pdb=pdb_name)

1 H-Adding task complete.


In [5]:
energy = ForceField(system, 'AMBER.FF14SB')

In [6]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)

In [7]:
md = Sponge(system, energy, min_opt)

In [8]:
run_info = RunInfo(10)
md.run(500, callbacks=[run_info])

[WARNING] KERNEL(21557,7ff8eca4f740,python):2022-08-15-14:33:46.265.785 [mindspore/ccsrc/plugin/device/gpu/kernel/cuda_impl/cuda_class/cuda_class_common.h:55] CalShapesSizeInBytes] For 'Neg', the shapes[0] is ( )


Step: 0, E_pot: 141277.78, 
Step: 10, E_pot: 96736.516, 
Step: 20, E_pot: 75656.49, 
Step: 30, E_pot: 62071.25, 
Step: 40, E_pot: 52376.703, 
Step: 50, E_pot: 45027.336, 
Step: 60, E_pot: 39223.355, 
Step: 70, E_pot: 34505.496, 
Step: 80, E_pot: 30588.402, 
Step: 90, E_pot: 27282.945, 
Step: 100, E_pot: 24456.771, 
Step: 110, E_pot: 22013.08, 
Step: 120, E_pot: 19878.992, 
Step: 130, E_pot: 17998.312, 
Step: 140, E_pot: 16327.074, 
Step: 150, E_pot: 14830.457, 
Step: 160, E_pot: 13480.786, 
Step: 170, E_pot: 12255.753, 
Step: 180, E_pot: 11137.305, 
Step: 190, E_pot: 10110.746, 
Step: 200, E_pot: 9163.998, 
Step: 210, E_pot: 8287.03, 
Step: 220, E_pot: 7471.5605, 
Step: 230, E_pot: 6710.5723, 
Step: 240, E_pot: 5998.165, 
Step: 250, E_pot: 5329.3, 
Step: 260, E_pot: 4699.6875, 
Step: 270, E_pot: 4105.591, 
Step: 280, E_pot: 3543.8271, 
Step: 290, E_pot: 3011.5645, 
Step: 300, E_pot: 2506.3652, 
Step: 310, E_pot: 2026.0898, 
Step: 320, E_pot: 1568.8037, 
Step: 330, E_pot: 1132.8066, 
St

In [9]:
vgen = VelocityGenerator(300)
velocity = vgen(system.coordinate.shape, system.atom_mass)

[WARNING] KERNEL(21557,7ff8eca4f740,python):2022-08-15-14:34:11.351.015 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel_factory.cc:93] ReducePrecision] Kernel [StandardNormal] does not support int64, cast input 0 to int32.
[WARNING] PRE_ACT(21557,7ff8eca4f740,python):2022-08-15-14:34:11.352.206 [mindspore/ccsrc/plugin/device/gpu/optimizer/reduce_precision_fusion.cc:84] Run] Reduce precision for [StandardNormal] input 0


In [10]:
opt = DynamicUpdater(system,
                     integrator=VelocityVerlet(system),
                     thermostat=Langevin(system, 300),
                     time_step=1e-3,
                     velocity=velocity)

In [11]:
md = Sponge(system, energy, min_opt)

In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b06.h5md', save_freq=10, write_velocity=True, write_force=True)

In [13]:
md.change_optimizer(opt)
md.run(2000, callbacks=[run_info, cb_h5md])

[WARNING] KERNEL(21557,7ff8eca4f740,python):2022-08-15-14:34:17.508.028 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel_factory.cc:93] ReducePrecision] Kernel [StandardNormal] does not support int64, cast input 0 to int32.
[WARNING] PRE_ACT(21557,7ff8eca4f740,python):2022-08-15-14:34:17.541.462 [mindspore/ccsrc/plugin/device/gpu/optimizer/reduce_precision_fusion.cc:84] Run] Reduce precision for [StandardNormal] input 0


Step: 0, E_pot: -4076.3867, E_kin: 9410.519, E_tot: 5334.132, Temperature: 298.4771
Step: 10, E_pot: -3697.8032, E_kin: 7578.38, E_tot: 3880.5767, Temperature: 240.36646
Step: 20, E_pot: -4086.9902, E_kin: 9538.316, E_tot: 5451.326, Temperature: 302.53052
Step: 30, E_pot: -3298.1406, E_kin: 9015.568, E_tot: 5717.4277, Temperature: 285.95032
Step: 40, E_pot: -4622.5137, E_kin: 10523.611, E_tot: 5901.0977, Temperature: 333.78152
Step: 50, E_pot: -4311.3535, E_kin: 9763.19, E_tot: 5451.837, Temperature: 309.66296
Step: 60, E_pot: -4552.1377, E_kin: 9773.1875, E_tot: 5221.05, Temperature: 309.98004
Step: 70, E_pot: -4262.422, E_kin: 9748.969, E_tot: 5486.547, Temperature: 309.21188
Step: 80, E_pot: -3947.5078, E_kin: 9759.893, E_tot: 5812.385, Temperature: 309.55835
Step: 90, E_pot: -4503.5645, E_kin: 10081.858, E_tot: 5578.294, Temperature: 319.77026
Step: 100, E_pot: -4601.1353, E_kin: 10137.48, E_tot: 5536.345, Temperature: 321.53445
Step: 110, E_pot: -4396.6143, E_kin: 9751.86, E_tot: 